# Aggregates in MongoDB

In MongoDB, aggregations are handled via the so-called "aggregation pipeline". 


For each aggregation pipeline, there is a match stage and a group stage. 
These two stages, when combined together cleverly, 
allow for powerful aggregate expressions. 

Documentation for the aggregate pipeline can be found at: https://docs.mongodb.com/v3.4/core/aggregation-pipeline/ which in turn has several related links. 
If you are looking for more information, we suggest reading these pages. 

The examples we are providing below are largely based on the exmaples in the documentation, 
adapted for our data and extended where relevant. 

In [ ]:
from bson.son import SON # for ordering by more than one column
from pprint import pprint
from pymongo import MongoClient
import pymongo

# Initialize a Mongo Client
#################################################
# Update UPDATE-ME in the connection code
#################################################
# Client 1 - mongodb-1.dsa.missouri.edu
# Client 2 - mongodb-2.dsa.missouri.edu
# Client 3 - mongodb-3.dsa.missouri.edu
# Client 4 - mongodb-4.dsa.missouri.edu
#################################################
#
client = MongoClient('UPDATE-ME',
                     username='ml_small_reader',
                     password='mlsmall.read',
                     authSource='ml_small')
# retrieve the ml_small database from the connection
db = client.ml_small

### Number of movies released per year

Let's jump into the aggregate pipeline with a few examples, 
beginning with the number of movies released each year, ordered by date ascending.

In [ ]:
pipeline = [
    {"$group" : { "_id" : {"$year": "$release_date"} , "count" : { "$sum" : 1 }}},
    {"$sort": { "_id": 1 }}
]
for doc in db.item.aggregate(pipeline):
    print(doc)

### Number of movies released per year: Top 10 years, ordered by count.


In [ ]:
pipeline = [
    {"$group" : { "_id" : {"$year": "$release_date"} , "count" : { "$sum" : 1 }}},
    {"$sort": { "count": -1 }},
    {"$limit": 10}
]

for doc in db.item.aggregate(pipeline):
    print(doc)

**The above query: **
 1. Groups the information based on the year of release, calculated using `$year` command and the date.  
 1. Creates a new field called "count" whose value comes from the `$sum` directive
 1. Sort the rows based on the count field in descending order
 1. Limit the results to 10 documents.

### Number of movies released for each month, sorted most to fewest


In [ ]:
pipeline = [
    {"$group" : { "_id" : {"$month": "$release_date"} , "count" : { "$sum" : 1 }}},
    {"$sort": { "count": -1 }}
]

for doc in db.item.aggregate(pipeline):
    print(doc)

As you can see, most of the documents in our dataset were released in Janurary. 
Now is a good time to highlight the fact that our data may not represent the reality.
Seeing such a huge bias for the first month in the year suggest that movies which the data
collectors could not find a specific release date for probably ended up with Janurary as the default date. 

Let's explore this idea by looking at the same breakdown, but based on day.

In [ ]:
pipeline = [
    {"$group" : { "_id" : {"$dayOfMonth": "$release_date"} , "count" : { "$sum" : 1 }}},
    {"$sort": { "count": -1 }}
]

for doc in db.item.aggregate(pipeline):
    print(doc)

With 1121 entries having their release date by the first day of the month, 
it would seem our suspicions are founded. 

As just one more example of aggregation with filtering, 
let's count the number of movies per relese day, where the release month is January.

In [ ]:
pipeline = [
    {"$project": {
        "_id": {"$dayOfMonth": "$release_date"}, 
        "monthReleased": {"$month": "$release_date"}
    }},
    {"$match": {"monthReleased": 1}},
    {"$group" : { "_id" : "$_id" , "count" : { "$sum" : 1 }}},
    {"$sort": { "count": -1 }}
]

for doc in db.item.aggregate(pipeline):
    print(doc)

**The above pipeline:**
 1. Sets the \_id field to the result of the `$dayOfMonth` operation on the `$releaseDate` field.
 1. Calculates a new value to be used for matching, to filter our documents to only those where the match condition is met. This value is the month of the release date.
 1. Limits our results using `$match` to those documents that meet the filter.
 1. Groups the remaining documents by their release day (of the month) and adds a count field. 
 1. Sorts the collection in descending order based on the count field.

There are even more powerful things you can do with the aggregation pipeline.
In fact, if you can think of something you want to know about the data, 
there is almost definitely a way to get the answer using the aggregate pipeline. 

## <span style="background:yellow">Your Turn</span>

Complete following lab work to gain hands-on experience with the Aggregate Pipeline.
**Make sure that you have ran the above cells, and have a connection to the DB!**

### Task 1

#### Get counts of movies based on combinations of (Action, Adventure) genres. 
This means that your result should have 4 documents.

In [ ]:
# Add your code below
# -------------------------



pipeline = [
    {"$group" : { 
            "_id" : {
                "Action": "$Action", "Adventure": "$Adventure"
            }, 
            "count" : { "$sum" : 1 }
    }}
]

for doc in db.item.aggregate(pipeline):
    print(doc)

### Task 2

#### Get counts of reviewers based on gender.
Use the `user` collection. Explore this collection using find_one() if necessary.

In [ ]:
# Add your code below
# -------------------------

pipeline = [
    {"$group": {
        "_id": "$gender",
        "count" : {"$sum": 1}
    }}
]
for doc in db.user.aggregate(pipeline):
    print(doc)






### Task 3

#### Get counts of reviewers, grouped by gender and age.
To sort by both gender and age, you will need to use __[SON](https://stackoverflow.com/questions/36566166/sort-the-result-from-a-pymongo-query#36566229)__.

In [ ]:
# Add your code below
# -------------------------

pipeline = [
    {"$group": {
        "_id": {"gender": "$gender", "age": "$age"},
        "count" : {"$sum": 1}
    }},
    {"$sort": SON([("_id.gender", 1), ("_id.age", 1)])}
]
for doc in db.user.aggregate(pipeline):
    print(doc)




In [ ]:
# Be sure to run this cell when you are finished. Thank you.
client.close()

# Save your notebook, then `File > Close and Halt`

---